<a href="https://colab.research.google.com/github/vishwa-R13/Cinema_ticket/blob/main/recommend.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics.pairwise import cosine_similarity

df = pd.read_csv('tasks.csv')

df['Cuisines'] = df['Cuisines'].fillna('Unknown')
df['Locality'] = df['Locality'].fillna('Unknown')
df['Average Cost for two'] = df['Average Cost for two'].fillna(df['Average Cost for two'].median())
df['Aggregate rating'] = df['Aggregate rating'].fillna(df['Aggregate rating'].mean())

features_df = df[['Cuisines', 'Locality', 'Average Cost for two', 'Aggregate rating']]

encoder = OneHotEncoder(handle_unknown='ignore')
encoded_cats = encoder.fit_transform(features_df[['Cuisines', 'Locality']])

num_features = features_df[['Average Cost for two', 'Aggregate rating']].values

X = np.hstack([encoded_cats.toarray(), num_features])

def build_user_vector(preferred_cuisine, preferred_locality, max_cost, min_rating):
    # Make a dummy user profile
    user_df = pd.DataFrame([{
        'Cuisines': preferred_cuisine,
        'Locality': preferred_locality,
        'Average Cost for two': max_cost,
        'Aggregate rating': min_rating,
    }])
    encoded_cats_user = encoder.transform(user_df[['Cuisines', 'Locality']]).toarray()
    num_feats_user = user_df[['Average Cost for two', 'Aggregate rating']].values
    return np.hstack([encoded_cats_user, num_feats_user])

def recommend_restaurants(preferred_cuisine, preferred_locality, max_cost, min_rating, top_n=10):
    user_vector = build_user_vector(preferred_cuisine, preferred_locality, max_cost, min_rating)
    sims = cosine_similarity(user_vector, X)[0]
    top_indices = sims.argsort()[::-1][:top_n]

    return df.iloc[top_indices][['Restaurant Name', 'Cuisines', 'Locality', 'Aggregate rating', 'Average Cost for two']]

if __name__ == "__main__":
    # Example preferences
    preferred_cuisine = 'North Indian'
    preferred_locality = 'Indiranagar'
    max_cost = 800
    min_rating = 4.0

    print(f"🔍 Recommendations for cuisine={preferred_cuisine}, locality={preferred_locality}, max_cost={max_cost}, min_rating={min_rating}\n")
    recommendations = recommend_restaurants(preferred_cuisine, preferred_locality, max_cost, min_rating, top_n=10)
    print(recommendations)


🔍 Recommendations for cuisine=North Indian, locality=Indiranagar, max_cost=800, min_rating=4.0

                 Restaurant Name                Cuisines  \
9173              Barbeque Ville            North Indian   
731             Glen's Bakehouse  Bakery, Desserts, Cafe   
9148                #Dilliwaala6            North Indian   
4444                      Chowki            North Indian   
2224       Indian Bistro Company            North Indian   
4544  Chawla's Tandoori Junction            North Indian   
2543               Eastern Spice            North Indian   
8255                       Noshh            North Indian   
2511                Tandooriwala            North Indian   
7451           Galina Restaurant            North Indian   

                                    Locality  Aggregate rating  \
9173                                   Wakad               4.4   
731                              Indiranagar               4.0   
9148                              White Town 